In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os

os.environ['__NV_PRIME_RENDER_OFFLOAD'] = '1'
os.environ['__GLX_VENDOR_LIBRARY_NAME'] = 'nvidia'
os.environ['MUJOCO_GL'] = 'egl'

# Tell XLA to use Triton GEMM, this improves steps/sec by ~30% on some GPUs
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags

from functools import partial
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.agents.ppo import train as ppo
import wandb


import jax
import mediapy as media
from randomize import domain_randomize
np.set_printoptions(precision=3, suppress=True, linewidth=100)


In [2]:

import balance
env = balance.G1Env()
eval_env = balance.G1Env()
env_cfg = balance.default_config()

# jit_reset = jax.jit(env.reset)
# jit_step = jax.jit(env.step)
# state = jit_reset(jax.random.PRNGKey(0))
# rollout = [state]

# f = 0.5

# for i in tqdm(range(200)):
#   action = []
#   for j in range(env.action_size):

#     if env.mj_model.actuator(j).name == "right_knee_joint" or env.mj_model.actuator(j).name == "left_shoulder_roll_joint":
#       value = jp.sin(
#             state.data.time * 2 * jp.pi * f 
#         ) * 1.
#     else:
#       value = 0.
      
#     action.append( value)
#   action = jp.array(action)
#   state = jit_step(state, action)
#   rollout.append(state)
# frames = env.render(rollout, camera="track")


In [3]:

from datetime import datetime

env_name = "g1_balance"
now = datetime.now()
timestamp = now.strftime("%Y%m%d-%H%M%S")
exp_name = f"{env_name}-{timestamp}"
import os

ckpt_path = os.path.abspath(os.path.join(".", "checkpoints", exp_name))
os.makedirs(ckpt_path, exist_ok=True)
print(f"Checkpoint path: {ckpt_path}")# media.show_video(frames, fps=1.0 / env.dt)


wandb.init(project="mjxrl", config=env_cfg)
wandb.config.update({
    "env_name": env_name,
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Checkpoint path: /home/logan/Projects/g1_mjx_helloworld/balance_experiment/checkpoints/g1_balance-20250414-221130


wandb: Currently logged in as: jloganolson (jloganolson-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from ml_collections import config_dict
# ppo_params= config_dict.create(
#     num_timesteps=1_000_000,
#     num_evals=10,
#     reward_scaling=10.0,
#     episode_length=env_cfg.episode_length,
#     normalize_observations=True,
#     action_repeat=1,
#     unroll_length=30,
#     num_minibatches=32,
#     num_updates_per_batch=16,
#     discounting=0.995,
#     learning_rate=1e-3,
#     entropy_cost=1e-2,
#     num_envs=2048,
#     batch_size=1024,
# )

from ml_collections import config_dict
ppo_params= config_dict.create(
    num_timesteps=100_000_000,
    reward_scaling=10.0,
    episode_length=env_cfg.episode_length,
    normalize_observations=True,
    action_repeat=1,
    unroll_length=30,
    num_minibatches=32,
    num_updates_per_batch=16,
    discounting=0.995,
    learning_rate=1e-3,
    entropy_cost=1e-2,
    num_envs=2048,
    batch_size=1024,
    num_evals=0,
    log_training_metrics=True,
    network_factory=config_dict.create(
        policy_hidden_layer_sizes=(512, 256, 128),
        value_hidden_layer_sizes=(512, 256, 128),
        policy_obs_key="state",
        value_obs_key="privileged_state",
    )
)
x_data, y_data, y_dataerr = [], [], []
times = [datetime.now()]


def progress_cli(num_steps, metrics):
  """Prints progress metrics to the console, including all available metrics."""

  wandb.log(metrics, step=num_steps)

  # Print the current step number
  print(f"Step: {num_steps}")

  # Print the entire metrics dictionary for debugging
  print("Metrics:", metrics)

  # You can add a separator for clarity if you run this multiple times
  print("-" * 20)

ppo_training_params = dict(ppo_params)
network_factory = ppo_networks.make_ppo_networks
if "network_factory" in ppo_params:
  del ppo_training_params["network_factory"]
  network_factory = partial(
      ppo_networks.make_ppo_networks,
      **ppo_params.network_factory
  )
from mujoco_playground import wrapper

train_fn = partial(
    ppo.train, **dict(ppo_training_params),
    network_factory=network_factory,
    progress_fn=progress_cli,
    randomization_fn=domain_randomize,
    wrap_env_fn=wrapper.wrap_for_brax_training,
    save_checkpoint_path=ckpt_path  
)

In [6]:
make_inference_fn, params, metrics = train_fn(
    environment=env,
    eval_env=eval_env
)
# print(f"time to jit: {times[1] - times[0]}")
# print(f"time to train: {times[-1] - times[1]}")
# # ~6m11s

/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/jax/_src/interpreters/xla.py:132: RuntimeWarning: overflow encountered in cast
  return np.asarray(x, dtypes.canonicalize_dtype(x.dtype))


Step: 983040
Metrics: {'episode/length': np.float64(46.89), 'episode/reward/dof_pos_limits': np.float64(-1.1098488987237216), 'episode/reward/height': np.float64(3.3354951761243865), 'episode/reward/orientation': np.float64(12.38266056060791), 'episode/reward/pose': np.float64(-5.648868931531906), 'episode/sum_reward': np.float64(0.17918875068426132)}
--------------------


E0414 22:13:30.378990   45557 pjrt_stream_executor_client.cc:3077] Execution of replica 0 failed: INTERNAL: CustomCall failed: CpuCallback error: Traceback (most recent call last):
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/runpy.py", line 196, in _run_module_as_main
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/runpy.py", line 86, in _run_code
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/as

XlaRuntimeError: INTERNAL: CustomCall failed: CpuCallback error: Traceback (most recent call last):
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/runpy.py", line 196, in _run_module_as_main
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/runpy.py", line 86, in _run_code
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once
  File "/home/logan/miniconda3/env

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x795e8eb6eda0>> (for post_run_cell), with arguments args (<ExecutionResult object at 795e8d7bba30, execution_count=6 error_before_exec=None error_in_exec=INTERNAL: CustomCall failed: CpuCallback error: Traceback (most recent call last):
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/runpy.py", line 196, in _run_module_as_main
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/runpy.py", line 86, in _run_code
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
  File "/home/logan/miniconda3/envs/g1/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)
jit_inference_fn = jax.jit(make_inference_fn(params, deterministic=True))
rng = jax.random.PRNGKey(42)
rollout = []
n_episodes = 1

for _ in range(n_episodes):
  state = jit_reset(rng)
  rollout.append(state)
  for i in range(env_cfg.episode_length):
    act_rng, rng = jax.random.split(rng)
    ctrl, _ = jit_inference_fn(state.obs, act_rng)
    state = jit_step(state, ctrl)
    rollout.append(state)

render_every = 1
frames = env.render(rollout[::render_every])
rewards = [s.reward for s in rollout]
media.show_video(frames, fps=1.0 / env.dt / render_every)
# ~11s